In [4]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

In [3]:
class Net(nn.Module):

    def __init__(self, num_classes=28):
        """Defining the Constructor - define each layer to be used in our model"""
        # RGB images, so we have 3 input channels
        super(Net, self).__init__()

        # apply 12 filters in first conv. layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)

        # second conv. layer takes 12 input channels and generates 24 outputs
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3, stride=1, padding=1)

        # apply max pooling with kernel size 2
        self.pool = nn.MaxPool2d(kernel_size=2)

        # drop layer to delete 20% of features to help prevent overfitting
        self.drop = nn.Dropout2d(p=0.2)

        # 200x200 image tensors pooled twice with kernel size 2 -> feature tensors now 50x50, and 24 of them generated

        # flatten feature tensors to feed to fully-connected layer
        self.fc = nn.Linear(in_features=50*50*24, out_features=num_classes)

    def forward(self, x):
        """Pass data through layers defined in constructor"""
        #  use ReLU activation function after layer 1 (convolution 1 & pool)
        x = F.relu(self.pool(self.conv1(x)))

        # use ReLU activation function after layer 2
        x = F.relu(self.pool(self.conv2(x)))

        # select some features to drop to prevent overfitting
        x = F.dropout(self.drop(x), training=self.training)

        # flatten
        x = x.view(-1, 50*50*24)

        # feed to fully-connected layer to predict class
        x = self.fc(x)

        return torch.log_softmax(x, dim=1) # return class probabilities
